# Santander Customer Satisfaction
## Which customers are happy customers?

https://www.kaggle.com/competitions/santander-customer-satisfaction/overview

You are provided with an anonymized dataset containing a large number of numeric variables. The "TARGET" column is the variable to predict. It equals one for unsatisfied customers and 0 for satisfied customers.

The task is to predict the probability that each customer in the test set is an unsatisfied customer.

In [1]:
# Imports
import numpy as np
#from pyspark.ml import Pipeline
from pyspark.ml.feature import IDF
from pyspark.ml.feature import HashingTF, Tokenizer
#from pyspark.ml.classification import NaiveBayes, NaiveBayesModel
#from pyspark.ml.evaluation import MulticlassClassificationEvaluator

#VETOR DENSO
from pyspark.ml.linalg import Vectors
from pyspark.sql import Row

In [2]:
# Criando o Spark Context
sc = SparkContext.getOrCreate();

In [3]:
sc.setLogLevel("ERROR")
sc

<SparkContext master=local[*] appName=PySparkShell>

In [4]:
# Spark Session - usada quando se trabalha com Dataframes no Spark
spSession = SparkSession.builder\
.master("local")\
.appName("Projeto-Santander")\
.config("spark.executor.memory", "6g")\
.config("spark.driver.memory", "12g").getOrCreate()

# Carga do dataset e transformação dos dados

In [5]:
# Importando o dataset
dataset = spSession.read.options(header = 'True').csv("train.csv")
#dataset.printSchema()
type(dataset)

pyspark.sql.dataframe.DataFrame

In [ ]:
# Coloco o Dataframe em cache
dataset.cache()

In [6]:
# convertendo uma columna em numerico

from pyspark.sql.types import IntegerType, FloatType

#dataset.withColumn("ID",dataset.ID.cast(IntegerType())).printSchema()

In [7]:
cols = dataset.columns
#cols

#import col
from pyspark.sql.functions import col

#converto todas as colunas do dataframe para float
for col_name in cols:
    dataset = dataset.withColumn(col_name, col(col_name).cast('float'))
    
#treino.printSchema()

In [8]:
# Numero de linhas e colunas spark df
#dataset.count() #linhas
len(dataset.columns)

#dataset.show(n=1)

371

# Pre-processamento Vetor Denso

- Transformação em Vetor Denso
- Creação do Spark dataframe 
- Baseline do modelo em MLlib

reff https://stackoverflow.com/questions/45063591/creating-spark-dataframe-from-numpy-matrix

In [ ]:
#Transformo o Spark Dataframe em um array numpy para transformar em vetor denso
#dataset_array = np.array(dataset.rdd.collect())
#type(dataset_array)

#Função map do PYTHON para crear um df com o TARGET e VETOR com as variaveis preditoras
#df = map(lambda x : (int(x [-1]), Vectors.dense(x[1:370])), dataset_array)

#type(df)

#dataset_vector = spark.createDataFrame(df,schema=["label", "features"])
#dataset_vector.cache()
#dataset_vector.printSchema()

# Pre-processamento - Vetor Sparse

- Transformação em Vetor Sparso


- Durante a fase de pre-processamento, transformo com sucesso o dataframe em um dataframe spark de 2 columnas label e Features (Feature em vetor denso) e ao criar o aplicar o modelo PCA ao dataframe tenho um estouro de memoria do Java

( java.lang.OutOfMemoryError : Java heap space )

Tentei configurar mais recursos na sessão do spark sem sucesso :(

- Lendo sobre o problema, analisei que a quantidade de columnas no dataframe era extremamente alta. Então foquei na criação de um vetor sparso para as 369 colunas do dataframe (todas menos a ID e TARGET) com ajuda da função VectorAssembler

https://stackoverflow.com/questions/32556178/create-labeledpoints-from-spark-dataframe-in-python

Como resultado utilizo a função LabeledPoint com o label + features (agora um vetor sparso) resolvendo o problema de performance no proximo paso (redução da dimensionalidade)

In [ ]:
from pyspark.ml.feature import VectorAssembler

dataset.printSchema()

In [ ]:
type(cols)
cols[1:370]

In [11]:
assembler = VectorAssembler(
    inputCols=cols[1:370],
    outputCol='features')

dataset_transformed = assembler.transform(dataset)
type(dataset_transformed)

pyspark.sql.dataframe.DataFrame

In [12]:
from pyspark.mllib.regression import LabeledPoint
from pyspark.sql.functions import col

(dataset_transformed.select(col("TARGET").alias("label"), col("features"))
  .rdd
  .map(lambda row: LabeledPoint(row.label, row.features)))

PythonRDD[15] at RDD at PythonRDD.scala:53

In [ ]:
dataset_transformed.take(2)

In [13]:
type(dataset_transformed)

pyspark.sql.dataframe.DataFrame

In [14]:
# Renomeia a coluna TARGET para label (requerido por SparkML Lib)
# Note that withColumnRenamed function returns a new DataFrame and doesn’t modify the current DataFrame.

dataset_transformed = dataset_transformed.withColumnRenamed("TARGET","label")

# Analise distribuição da variavel target

In [15]:
dataset_transformed.groupBy("label").count().show()

+-----+-----+
|label|count|
+-----+-----+
|  1.0| 3008|
|  0.0|73012|
+-----+-----+



É necessario o balanceamento das classes preditoras.

reff https://medium.com/@junwan01/oversampling-and-undersampling-with-pyspark-5dbc25cdf253

In [ ]:
# Undersample da classe de clientes satisfeitos

In [16]:
# Extratifico uma amostra de 5% dos clientes satisfeitos ( n_clientes_insatisfeitos / n_clientes_satisfeitos )
sample_satisfied_clients = dataset_transformed.sample(False, 0.05)
sample_satisfied_clients.count()

3785

In [17]:
# Seleciono todos os registros de clientes insatisfeitos
sum_unsatisfied_clients = dataset_transformed.where(col("label")==1)
sum_unsatisfied_clients.count()

3008

In [18]:
# Crio novo dataframe com a soma da amostra extratificada + clientes insatisfeitos
dataset_transformed_balanced = sample_satisfied_clients.unionAll(sum_unsatisfied_clients)
dataset_transformed_balanced.count()

6793

In [19]:
# Shufle o dataframe balanceado
from pyspark.sql.functions import rand

dataset_transformed_balanced = dataset_transformed_balanced.orderBy(rand())

# Random Forest com dados normalizados

baseline para random forest

É um algoritmo de Método Ensemble. Um modelo de Random Forest constrói diversos modelos e cada modelo é usado para prever resultados de forma individual. Uma votação é feita pelo Random Forest paar escolher o melhor modelo


Vantagens

    - Normalmente oferece boa acurácia
    - Eficiente com muitas variáveis preditoras
    - Funciona muito bem de forma paralelizada
    - Excelente com valores missing

Desvantagens

    - Mais lento
    - Bias podem ocorrer com frequência

# Normalização
 

In [20]:
from pyspark.ml.feature import MinMaxScaler

In [22]:
dataset_transformed_balanced.select("features","label").limit(15).show()

+--------------------+-----+
|            features|label|
+--------------------+-----+
|(369,[0,1,12,13,1...|  0.0|
|(369,[0,1,23,24,6...|  0.0|
|(369,[0,1,23,24,6...|  0.0|
|(369,[0,1,23,62,7...|  1.0|
|(369,[0,1,23,24,6...|  0.0|
|(369,[0,1,23,62,7...|  1.0|
|(369,[0,1,23,62,8...|  1.0|
|(369,[0,1,3,4,10,...|  1.0|
|(369,[0,1,3,4,10,...|  1.0|
|(369,[0,1,13,16,2...|  0.0|
|(369,[0,1,23,62,7...|  1.0|
|(369,[0,1,23,62,7...|  1.0|
|(369,[0,1,3,4,10,...|  1.0|
|(369,[0,1,23,62,7...|  0.0|
|(369,[0,1,3,4,10,...|  0.0|
+--------------------+-----+



In [25]:
# Cria o scaler
scaler = MinMaxScaler(inputCol = "features", outputCol = "scaled_features")

In [27]:
# Crio novo dataset de treino com as colunas targets e features

#dataset_completo = dataset_transformed_balanced.select("features","label")

#ou

#seleciono todas as variaveis preditoras sem a variavel label
dataset_completo = dataset_transformed_balanced.select([c for c in dataset_transformed_balanced.columns if c in {'features', 'label'}])
#dataset_completo = dataset_transformed_balanced.select([c for c in dataset_transformed_balanced.columns if c not in {'label'}])

dataset_completo.printSchema()

root
 |-- label: float (nullable = true)
 |-- features: vector (nullable = true)



In [28]:
# Fit nos dados de treino
scalerModel = scaler.fit(dataset_completo)

In [27]:
# Fit e transform nos dados de treino 
dataset_completo_scaled = scalerModel.transform(dataset_completo)

In [ ]:
dataset_completo_scaled.select("features", "scaled_features").show(10, truncate = False)

In [29]:
dataset_completo_scaled.printSchema()

root
 |-- label: float (nullable = true)
 |-- features: vector (nullable = true)
 |-- scaled_features: vector (nullable = true)



Divisão do dataset entre treino e teste

In [30]:
# Dados de Treino e de Teste
(dataset_treino_scaled, dataset_teste_scaled) = dataset_completo_scaled.randomSplit([0.7, 0.3])

In [31]:
dataset_treino_scaled.count()

4801

In [32]:
dataset_teste_scaled.count()

1994

 Machine learning 


In [33]:
from pyspark.ml.feature import StringIndexer
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [34]:
# Criando o objeto
rfClassifer = RandomForestClassifier(labelCol = "label", featuresCol = "scaled_features")

In [35]:
# Treinando o objeto e criando o modelo
modelo_rf_scaled = rfClassifer.fit(dataset_treino_scaled)

In [36]:
# Previsões com dados de teste
predictions = modelo_rf_scaled.transform(dataset_teste_scaled)

In [37]:
#type(predictions)
predictions

DataFrame[label: float, features: vector, scaled_features: vector, rawPrediction: vector, probability: vector, prediction: double]

In [38]:
predictions.select("label","features","scaled_features","prediction").show(5)

+-----+--------------------+--------------------+----------+
|label|            features|     scaled_features|prediction|
+-----+--------------------+--------------------+----------+
|  0.0|(369,[0,1,2,3,4,1...|[0.99980403860439...|       0.0|
|  0.0|(369,[0,1,2,3,4,1...|[0.99980403860439...|       0.0|
|  0.0|(369,[0,1,2,3,4,1...|[0.99980403860439...|       0.0|
|  0.0|(369,[0,1,2,3,4,1...|[0.99980403860439...|       0.0|
|  0.0|(369,[0,1,2,3,4,1...|[0.99980403860439...|       0.0|
+-----+--------------------+--------------------+----------+
only showing top 5 rows



In [39]:
# Avaliando a acurácia
evaluator_1 = MulticlassClassificationEvaluator(predictionCol = "prediction", 
                                              labelCol = "label", 
                                              metricName = "accuracy")

evaluator_1.evaluate(predictions) 

0.7101303911735206

In [40]:
# Confusion Matrix
predictions.groupBy("label", "prediction").count().show()

+-----+----------+-----+
|label|prediction|count|
+-----+----------+-----+
|  1.0|       0.0|  291|
|  0.0|       0.0|  770|
|  1.0|       1.0|  646|
|  0.0|       1.0|  287|
+-----+----------+-----+



# Random Forest com Redução da dimensionalidade

- transformação da columna features em 5 grupos

reff https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.ml.feature.PCA.html

Caso faça a redução da dimensionalidade, não É necessario normalizar os dados. 

PELOS MEUS TESTES, O RESULTADO FINAL DA REDUÇÃO DA DIMENSIONALIDE NOS DADOS NORMALIZADOS E NÃO NORMALIZADOS FORAM IGUAIS.

In [41]:
from pyspark.ml.feature import PCA

In [42]:
# Aplicando Redução de Dimensionalidade com PCA
PCA = PCA(k = 5, inputCol = "features", outputCol = "pcaFeatures")

In [44]:
pcaModel = PCA.fit(dataset_transformed_balanced)

In [45]:
pcaResult = pcaModel.transform(dataset_transformed_balanced).select("label","pcaFeatures")
pcaResult.show(truncate = False)

+-----+--------------------------------------------------------------------------------------------------------+
|label|pcaFeatures                                                                                             |
+-----+--------------------------------------------------------------------------------------------------------+
|0.0  |[-1.7986253794612332,-0.15782841878570628,0.43202883966663497,-0.1729192953775434,-0.4501472322512038]  |
|0.0  |[-1.4875370518758118,-0.13639563707358918,0.3538555883967369,-0.15277872908572937,-0.3722199402571497]  |
|1.0  |[-3.07792721855811,-0.2718147294650136,0.737092763970423,-0.297476186295972,-0.7720484638587527]        |
|0.0  |[-1.798625381747714,-0.15782841982760096,0.432028836570613,-0.1729192982915344,-0.4501472289860694]     |
|0.0  |[-3.596132480004754,-0.3155588861200953,0.8637888005904967,-0.3457312970512437,-0.9000145140729922]     |
|1.0  |[-0.8003201618628747,-0.07022758316504736,0.19223661463847522,-0.07694242576135436,-0.200

Divisão do dataset entre treino e teste

In [46]:
# Dados de Treino e de Teste
(dataset_treino_PCA, dataset_teste_PCA) = pcaResult.randomSplit([0.7, 0.3])

In [47]:
dataset_treino_PCA.count()

4805

In [48]:
dataset_teste_PCA.count()

1990

Treinamento do modelo



In [49]:
# Criando o objeto
rfClassifer = RandomForestClassifier(labelCol = "label", featuresCol = "pcaFeatures")

In [50]:
# Treinando o objeto e criando o modelo PCA
modelo_rf_pca = rfClassifer.fit(pcaResult)

In [51]:
# Previsões com dados de teste
predictions = modelo_rf_pca.transform(dataset_teste_PCA)

In [52]:
predictions.select("label","pcaFeatures","prediction").show(5)

+-----+--------------------+----------+
|label|         pcaFeatures|prediction|
+-----+--------------------+----------+
|  0.0|[-1.4111606528473...|       0.0|
|  0.0|[-1.4111606527494...|       0.0|
|  0.0|[-16.132720749023...|       0.0|
|  0.0|[-11.435069227612...|       0.0|
|  0.0|[-10.668093310802...|       0.0|
+-----+--------------------+----------+
only showing top 5 rows



In [53]:
# Avaliando a acurácia
evaluator_2 = MulticlassClassificationEvaluator(predictionCol = "prediction", 
                                              labelCol = "label", 
                                              metricName = "accuracy")

evaluator_2.evaluate(predictions)

0.6115577889447236

In [54]:
# Confusion Matrix
predictions.groupBy("label", "prediction").count().show()

+-----+----------+-----+
|label|prediction|count|
+-----+----------+-----+
|  1.0|       0.0|  513|
|  0.0|       0.0|  824|
|  1.0|       1.0|  393|
|  0.0|       1.0|  260|
+-----+----------+-----+



 # Pre-processamento dados de teste
Performance do modelo baseline com os dados de teste

In [55]:
teste = spSession.read.options(header = 'True').csv("test.csv")
#teste.show (5)

In [56]:
cols = teste.columns

In [57]:
# Converte todas as colunas do dataset em float
for col_name in cols:
    teste = teste.withColumn(col_name, col(col_name).cast('float'))

#dados_prod = teste.select([x for x in teste.columns if x !='ID' ])


In [ ]:
#len(teste.columns)
#len(dados_teste.columns)

teste.printSchema()

In [ ]:
assembler = VectorAssembler(
    inputCols=cols[1:370], #excluindo a coluna id cols[1:]
    outputCol='features')

transformed_teste = assembler.transform(teste)
transformed_teste.printSchema()

In [74]:
# Crio novo dataset com os IDs e Features para o processo de normalização
dataset_prod_completo = transformed_teste.select("ID","features")
len(dataset_prod_scaled.columns)

2

In [87]:
# normalizo os dados de produção (mesma transformação aplicada aos dados de treino)
dataset_prod_scaled = scalerModel.transform(dataset_prod_completo)
dataset_prod_scaled.show(5)

+---+--------------------+--------------------+
| ID|            features|     scaled_features|
+---+--------------------+--------------------+
|2.0|(369,[0,1,23,24,6...|[0.99980403860439...|
|5.0|(369,[0,1,23,24,6...|[0.99980403860439...|
|6.0|(369,[0,1,12,13,1...|[0.99980403860439...|
|7.0|(369,[0,1,23,62,7...|[0.99980403860439...|
|9.0|(369,[0,1,23,24,6...|[0.99980403860439...|
+---+--------------------+--------------------+
only showing top 5 rows



In [78]:
#transformed_teste.printSchema()
dataset_prod_scaled.printSchema()

root
 |-- ID: float (nullable = true)
 |-- features: vector (nullable = true)
 |-- scaled_features: vector (nullable = true)



In [81]:
# Seleciona a coluna resultante
dataset_prod_scaled_final = dataset_prod_scaled.select("ID","scaled_features")

# Renomeia a coluna (requerimento do MLlib)
#dataset_prod_scaled_final = dataset_prod_scaled_final.withColumnRenamed("scaled_features","features")

dataset_prod_scaled_final.printSchema()

root
 |-- ID: float (nullable = true)
 |-- scaled_features: vector (nullable = true)



# Aplica o modelo de Random Forest com dados normalizados em produção



In [82]:
# Previsões com dados reais

predictions_prod = modelo_rf_scaled.transform(dataset_prod_scaled_final)

In [83]:
predictions_prod

DataFrame[ID: float, scaled_features: vector, rawPrediction: vector, probability: vector, prediction: double]

In [86]:
# Resultado das predições com dados de probabilidades para cada classe
predictions_prod.select("ID","prediction","probability" ).show(truncate = False)

+----+----------+----------------------------------------+
|ID  |prediction|probability                             |
+----+----------+----------------------------------------+
|2.0 |0.0       |[0.6679301180285775,0.3320698819714226] |
|5.0 |0.0       |[0.6755245825126829,0.3244754174873172] |
|6.0 |0.0       |[0.7627810779390559,0.23721892206094403]|
|7.0 |1.0       |[0.39546563297967846,0.6045343670203216]|
|9.0 |0.0       |[0.7692772951007705,0.23072270489922958]|
|11.0|1.0       |[0.327625645574113,0.6723743544258871]  |
|12.0|1.0       |[0.41198534188579145,0.5880146581142085]|
|15.0|1.0       |[0.30902609867408376,0.6909739013259163]|
|16.0|0.0       |[0.6815986750990803,0.31840132490091966]|
|17.0|0.0       |[0.6886792564610368,0.3113207435389633] |
|19.0|0.0       |[0.7172189924476335,0.28278100755236646]|
|21.0|0.0       |[0.7713473022080909,0.22865269779190905]|
|22.0|0.0       |[0.6977649857087154,0.30223501429128463]|
|24.0|0.0       |[0.7692772951007705,0.23072270489922958